In [1]:
import pandas as pd
import boto3
import io
import awswrangler as wr
from difflib import get_close_matches
import glob


def import_file_from_s3(s3_bucket, s3_key):
#     df = pd.read_parquet(s3_key)
    df = pd.read_parquet(s3_key)

    # s3 = boto3.client('s3')
    # obj = s3.get_object(Bucket=s3_bucket, Key=s3_key)
    # df = pd.read_csv(io.BytesIO(obj['Body'].read()))
    return df


def run_athena_query(database , query):
    print("Executing Query: ", query, "\n On Database: ", database)
    try : 
        df_iter = wr.athena.read_sql_query(
                sql = query,
                database = database,
                ctas_approach=True,
                chunksize=True
        )
    
        result_df = pd.concat(df_iter)
        return result_df
    except Exception as e:
        raise Exception(e)

def get_brand_dictionary_from_athena(database, table):
    query = """SELECT * from {table}""".format(table,table)
    
    result_df = run_athena_query(database, query)
        
    return result_df

In [2]:
args = {'s3_bucket' : '/home/preacher/Bungee/bnt',
'brand_dict_database' : 'brand_dict_database',
'brand_dict_table_name' : 'brand_dict_table_name',
'bnt_file_key' : '/home/preacher/Bungee/bnt' }

In [3]:
s3_bucket = args['s3_bucket']
brand_dict_database = args['brand_dict_database']
brand_dict_table_name = args['brand_dict_table_name']
bnt_file_key = args['bnt_file_key']
s3_unmatched_brand_object_key = bnt_file_key+'/unmatched_brands/'

# brand_dict = get_brand_dictionary_from_athena(brand_dict_database, brand_dict_table_name)
unmatched_brands = import_file_from_s3(s3_bucket, s3_unmatched_brand_object_key)

clusters = []
leftover_brands = []


In [13]:
brand_list = unmatched_brands['crawled_name'].tolist()

list_size = len(brand_list)
processed = [False] * list_size
cluster_brand_lengs = 0
cluster_id = 1

In [4]:
unmatched_brands

,crawled_name
0,G.H. MUMM
1,18TH STREET GIFTS
2,AMPLE:N
3,St. Ives Rose and Argan Oil Smoothing Body Lot...
4,FCXJTU
...,...
658575,Inigo Creations
658576,VTech Treasure Seekers Pirate Ship
658577,Gorilla Case
658578,I LOVE MY PET


In [14]:
brand_dict = {k: v for v, k in enumerate(brand_list)}

In [6]:
brand_dict

{'G.H. MUMM': 0,
 '18TH STREET GIFTS': 1,
 'AMPLE:N': 2,
 'St. Ives Rose and Argan Oil Smoothing Body Lotion 21 oz': 3,
 'FCXJTU': 4,
 'Sveda': 5,
 'PetAbilities': 6,
 'HYOUBALA': 7,
 'Dunacifa Women Sweatshirt': 8,
 'BIOSOTA': 9,
 'Amici Cellars': 10,
 'Sundip': 11,
 'ALESATION BREWING CO': 12,
 'NATRABIO': 13,
 'XMANTSEA': 14,
 'Xlears': 15,
 "STOUFFER'S CLASSICS Lasagna with Meat & Sauce 10.5 oz. Box": 16,
 'PREPARED FOODS JUICE BAR': 17,
 'Ronoal': 18,
 'Summit||API': 19,
 'PHICOOL': 20,
 'ACT II Butter Lovers Microwave Popcorn 2.75 Oz 3 Ct': 21,
 'DazDalee': 22,
 'Equate Kids Broad Spectrum Sunscreen Spray': 23,
 'Fresh & Ready': 24,
 'GOOVITOR': 25,
 'Aroma Chips': 26,
 'gundlach bundschu': 27,
 'CADEFU': 28,
 'JUZZQNEW': 29,
 'WhimsicalDarkness': 30,
 'Qurdtt': 31,
 'TALLEY VINEYARDS': 32,
 'TONBGSEL': 33,
 'CALWISE SPIRITS CO': 34,
 'Misika': 35,
 'Little': 36,
 '(12 Pack) Fancy Feast Grain Free Pate Wet Cat Food': 37,
 'Cock Grease': 38,
 'I-MGAE-IN-AR': 39,
 'Duixinghas': 40,

In [7]:
len(brand_dict)

658580

In [15]:

for idx, brand in enumerate(brand_list):
    if brand != '':
        print("Processing: {} out of {}".format(idx, len(brand_list)))
        print("Forming string cluster for :", brand)
        cluster = get_close_matches( brand, brand_list, 10, 0.8)
        print("Cluster : ", cluster)

        if cluster is not None and len(cluster) > 1:
            for item in cluster:
                brand_list[brand_dict[item]] = ''
                temp_dict = {'cluster_id': cluster_id, 'brand': item}
                clusters.append(temp_dict)

            cluster_id = cluster_id + 1
        else:
            brand_list[brand_dict[item]] = ''
    else:
        print("Brand is none")
        

Processing: 0 out of 658580
Forming string cluster for : G.H. MUMM
Cluster :  ['G.H. MUMM', 'GH MUMM', 'G H MUMM']
Processing: 1 out of 658580
Forming string cluster for : 18TH STREET GIFTS
Cluster :  ['18TH STREET GIFTS']
Processing: 2 out of 658580
Forming string cluster for : AMPLE:N
Cluster :  ['AMPLE:N', 'AMPLE']
Processing: 3 out of 658580
Forming string cluster for : St. Ives Rose and Argan Oil Smoothing Body Lotion 21 oz
Cluster :  ['St. Ives Rose and Argan Oil Smoothing Body Lotion 21 oz']
Processing: 4 out of 658580
Forming string cluster for : FCXJTU
Cluster :  ['FCXJTU', 'CXTU']
Processing: 5 out of 658580
Forming string cluster for : Sveda
Cluster :  ['Sveda', 'Svedka', 'Seda', 'veeda', 'aveda', 'Sweda', 'Sedal', 'Saved', 'Dveda', 'Aveda']
Processing: 6 out of 658580
Forming string cluster for : PetAbilities
Cluster :  ['PetAbilities', 'Petities', 'Pastabilities']
Processing: 7 out of 658580
Forming string cluster for : HYOUBALA
Cluster :  ['HYOUBALA']
Processing: 8 out of

Cluster :  ['TEMEAYE']
Processing: 56 out of 658580
Forming string cluster for : QIDIAN
Cluster :  ['QIDIAN', 'WIDIAN', 'QIJIAN', 'JIDIAN', 'INDIAN', 'QDAN']
Processing: 57 out of 658580
Forming string cluster for : Airflo Fly Lines
Cluster :  ['Airflo Fly Lines']
Processing: 58 out of 658580
Forming string cluster for : Caframo Limited
Cluster :  ['Caframo Limited']
Processing: 59 out of 658580
Forming string cluster for : SafeTGard
Cluster :  ['SafeTGard', 'Safe-T-Gard', 'SafeGuard', 'Safe Guard']
Processing: 60 out of 658580
Forming string cluster for : Gefvjus
Cluster :  ['Gefvjus']
Processing: 61 out of 658580
Forming string cluster for : New Leaf
Cluster :  ['New Leaf', 'Neo Leaf', 'New Lee', 'Be Leaf']
Processing: 62 out of 658580
Forming string cluster for : TISSAGES DE BEAULIEU
Cluster :  ['TISSAGES DE BEAULIEU']
Processing: 63 out of 658580
Forming string cluster for : REBESCO
Cluster :  ['REBESCO', 'RESCO']
Processing: 64 out of 658580
Forming string cluster for : Pet Bereav

Cluster :  ['WIKpAEUG']
Processing: 116 out of 658580
Forming string cluster for : Renzhen
Cluster :  ['Renzhen', 'Renzhe', 'Renhe', 'zhenzhen', 'Shenzhen', 'Renzhong', 'Renzhe22', 'Reinzein']
Processing: 117 out of 658580
Forming string cluster for : Digital World
Cluster :  ['Digital World']
Processing: 118 out of 658580
Forming string cluster for : Buyal
Cluster :  ['Buyal', 'Buyalot', 'Kuyal', 'BuyAl', 'Buoya']
Processing: 119 out of 658580
Forming string cluster for : Sugar Bytes
Cluster :  ['Sugar Bytes', 'Sugar Eyes', 'Sugar Bee']
Processing: 120 out of 658580
Forming string cluster for : shortway brewing co.
Cluster :  ['shortway brewing co.', 'northway brewing co.', 'parkway brewing co.']
Processing: 121 out of 658580
Forming string cluster for : In-hand
Cluster :  ['In-hand', 'In hand']
Processing: 122 out of 658580
Forming string cluster for : Moongo Tool
Cluster :  ['Moongo Tool']
Processing: 123 out of 658580
Forming string cluster for : Kicitum
Cluster :  ['Kicitum']
Proc

Cluster :  ['Today Wall Calendar 2021', 'Today Wall Calendars 2021', 'Today Wall Calendrs 2021']
Processing: 179 out of 658580
Forming string cluster for : TEMPEA
Cluster :  ['TEMPEA', 'TEMPESTA', 'TEMPLE', 'TEPE', 'TEPA', 'TEMPLESPA', 'TEMP', 'TEME', 'TEMA']
Processing: 180 out of 658580
Forming string cluster for : 125-217 MD
Cluster :  ['125-217 MD', '125-227 MD', '125-254 MD', '125-233 MD', '125-224 MD', '125-223 MD', '125-206 MD', '125-102 MD']
Processing: 181 out of 658580
Forming string cluster for : Great Value Organic Ground Flax Seed
Cluster :  ['Great Value Organic Ground Flax Seed', 'Great Value Organic Whole Flax Seed', 'Great Value Organic Ground Black Pepper', 'Great Value Organic Ground Cloves', 'Great Value Organic Ground Cardamom', 'Great Value Organic Ground Allspice', 'Great Value Organic Ground Nutmeg', 'Great Value Organic Ground Ginger', 'Great Value Organic Fennel Seed', 'Great Value Organic Ground Coriander']
Processing: 182 out of 658580
Forming string cluster

Cluster :  ['Hop Water', 'Hop Wtr', 'Home Water', 'Hapi Water', 'Happy Water']
Processing: 239 out of 658580
Forming string cluster for : HFCCYBGS
Cluster :  ['HFCCYBGS']
Processing: 240 out of 658580
Forming string cluster for : Lidlife
Cluster :  ['Lidlife', 'Livlife', 'Llife', 'Wildlife', 'Lovilife', 'Litelife']
Processing: 241 out of 658580
Forming string cluster for : Muhunthan Thillai (Editor) David R. Moller (Editor) Keith C. Meyer (Editor)
Cluster :  ['Muhunthan Thillai (Editor) David R. Moller (Editor) Keith C. Meyer (Editor)']
Processing: 242 out of 658580
Forming string cluster for : BARCRAFT
Cluster :  ['BARCRAFT', 'BAR CRAFT', 'LABACRAFT', 'FAIRCRAFT', 'DEARCRAFT', 'BAKECRAFT', 'TACRAFT', 'BARRATT']
Processing: 243 out of 658580
Forming string cluster for : Hartz||Purina Pro Plan
Cluster :  ['Hartz||Purina Pro Plan', 'Hartz and Purina Pro Plan', 'EverRoot||Purina Pro Plan', 'Arf Pets||Purina Pro Plan', 'Virbac||Purina Pro Plan']
Processing: 244 out of 658580
Forming string

Cluster :  ['LLDHWX']
Processing: 304 out of 658580
Forming string cluster for : Rishi-doom(Fast Shipping 7-15 Days )
Cluster :  ['Rishi-doom(Fast Shipping 7-15 Days )']
Processing: 305 out of 658580
Forming string cluster for : SENSIBLE PORTIONS
Cluster :  ['SENSIBLE PORTIONS']
Processing: 306 out of 658580
Forming string cluster for : whimsyyy
Cluster :  ['whimsyyy']
Processing: 307 out of 658580
Forming string cluster for : Bentley Global Arts
Cluster :  ['Bentley Global Arts']
Processing: 308 out of 658580
Forming string cluster for : joly
Cluster :  ['joly', 'Ejoly', 'joy', 'jconly']
Processing: 309 out of 658580
Forming string cluster for : Wri Obb (Author)
Cluster :  ['Wri Obb (Author)', 'Erin Cobb (Author)', 'Erik Oberg (Author)']
Processing: 310 out of 658580
Forming string cluster for : invinsible besstore
Cluster :  ['invinsible besstore']
Processing: 311 out of 658580
Forming string cluster for : VASONNI
Cluster :  ['VASONNI']
Processing: 312 out of 658580
Forming string cl

Cluster :  ['Gardenburger']
Processing: 363 out of 658580
Forming string cluster for : BABAS BREW
Cluster :  ['BABAS BREW']
Processing: 364 out of 658580
Forming string cluster for : Alicacho
Cluster :  ['Alicacho', 'Alcachofa']
Processing: 365 out of 658580
Forming string cluster for : Teamy
Cluster :  ['Teamy', 'Teamoy', 'Team', 'Teamsky', 'Teamery', 'TeamSky', 'Tsemy', 'Tezam', 'Teday', 'Teami']
Processing: 366 out of 658580
Forming string cluster for : GYBYW
Cluster :  ['GYBYW']
Processing: 367 out of 658580
Forming string cluster for : black bird cider works
Cluster :  ['black bird cider works', 'blackbird cider works', 'Blackbird Cider Works', 'Black Bird Cider Works']
Processing: 368 out of 658580
Forming string cluster for : Bombril
Cluster :  ['Bombril']
Processing: 369 out of 658580
Forming string cluster for : Kriaa Goodness
Cluster :  ['Kriaa Goodness']
Processing: 370 out of 658580
Forming string cluster for : Brian Taylor (Author)
Cluster :  ['Brian Taylor (Author)', 'Bri

Cluster :  ['Bob Mackie']
Processing: 417 out of 658580
Forming string cluster for : Surefire
Cluster :  ['Surefire', 'SureFire', 'Purefire', 'Surfree', 'Sunfire']
Processing: 418 out of 658580
Forming string cluster for : peamonte
Cluster :  ['peamonte', 'veramonte', 'Veramonte']
Processing: 419 out of 658580
Forming string cluster for : ZHEN GUO
Cluster :  ['ZHEN GUO', 'ZHENGO', 'ZHENGAO', 'ZHENDUO']
Processing: 420 out of 658580
Forming string cluster for : Yat Ming
Cluster :  ['Yat Ming', 'Yu Ming', 'Yatming']
Processing: 421 out of 658580
Forming string cluster for : SHARBDA
Cluster :  ['SHARBDA', 'SHARD', 'SARDA', 'HARDA', 'SHARPBEA', 'CHARYBDA']
Processing: 422 out of 658580
Forming string cluster for : CHAI WALLAH
Cluster :  ['CHAI WALLAH']
Processing: 423 out of 658580
Forming string cluster for : EasyToys
Cluster :  ['EasyToys', 'Easytoy', 'EasyBoy']
Processing: 424 out of 658580
Forming string cluster for : GemeShou
Cluster :  ['GemeShou']
Processing: 425 out of 658580
Formi

KeyboardInterrupt: 

In [17]:
i = 0 
for brand in brand_list:
    if brand == '':
        i = i + 1
        print('empty brand')

print(i)

empty brand
empty brand
empty brand
empty brand
empty brand
empty brand
empty brand
empty brand
empty brand
empty brand
empty brand
empty brand
empty brand
empty brand
empty brand
empty brand
empty brand
empty brand
empty brand
empty brand
empty brand
empty brand
empty brand
empty brand
empty brand
empty brand
empty brand
empty brand
empty brand
empty brand
empty brand
empty brand
empty brand
empty brand
empty brand
empty brand
empty brand
empty brand
empty brand
empty brand
empty brand
empty brand
empty brand
empty brand
empty brand
empty brand
empty brand
empty brand
empty brand
empty brand
empty brand
empty brand
empty brand
empty brand
empty brand
empty brand
empty brand
empty brand
empty brand
empty brand
empty brand
empty brand
empty brand
empty brand
empty brand
empty brand
empty brand
empty brand
empty brand
empty brand
empty brand
empty brand
empty brand
empty brand
empty brand
empty brand
empty brand
empty brand
empty brand
empty brand
empty brand
empty brand
empty brand
empt

In [26]:
clusters

[{'id': 1, 'cluster': 'AMPLE:N'},
 {'id': 1, 'cluster': 'DAMPEN'},
 {'id': 1, 'cluster': 'MLEN'},
 {'id': 2, 'cluster': 'PetAbilities'},
 {'id': 2, 'cluster': 'Petities'},
 {'id': 2, 'cluster': 'PetUltimates'},
 {'id': 3, 'cluster': 'Dunacifa Women Sweatshirt'},
 {'id': 3, 'cluster': 'Dunacifa Women Shorts'},
 {'id': 4, 'cluster': 'Amici Cellars'},
 {'id': 4, 'cluster': 'Pino Cellars'},
 {'id': 4, 'cluster': 'America Gear'},
 {'id': 5, 'cluster': 'ALESATION BREWING CO'},
 {'id': 5, 'cluster': 'QUEST BREWING CO'},
 {'id': 5, 'cluster': 'LANSING BREWING CO'},
 {'id': 6, 'cluster': 'XMANTSEA'},
 {'id': 6, 'cluster': 'MASERA'},
 {'id': 6, 'cluster': 'MANTO'},
 {'id': 7,
  'cluster': "STOUFFER'S CLASSICS Lasagna with Meat & Sauce 10.5 oz. Box"},
 {'id': 7,
  'cluster': "STOUFFER'S CLASSICS Party Size Lasagna Italiano 90 oz. Box"},
 {'id': 8, 'cluster': 'Ronoal'},
 {'id': 8, 'cluster': 'ournal'},
 {'id': 8, 'cluster': 'Royali'},
 {'id': 9, 'cluster': 'PHICOOL'},
 {'id': 9, 'cluster': 'POROPL

In [15]:
dict_list

[{'id': 0, 'cluster': ['AMPLE:N', 'DAMPEN', 'MLEN']},
 {'id': 1, 'cluster': ['PetAbilities', 'Petities', 'PetUltimates']},
 {'id': 2, 'cluster': ['Dunacifa Women Sweatshirt', 'Dunacifa Women Shorts']},
 {'id': 3, 'cluster': ['Amici Cellars', 'Pino Cellars', 'America Gear']},
 {'id': 4,
  'cluster': ['ALESATION BREWING CO',
   'QUEST BREWING CO',
   'LANSING BREWING CO']},
 {'id': 5, 'cluster': ['XMANTSEA', 'MASERA', 'MANTO']},
 {'id': 6,
  'cluster': ["STOUFFER'S CLASSICS Lasagna with Meat & Sauce 10.5 oz. Box",
   "STOUFFER'S CLASSICS Party Size Lasagna Italiano 90 oz. Box"]},
 {'id': 7, 'cluster': ['Ronoal', 'ournal', 'Royali']},
 {'id': 8, 'cluster': ['PHICOOL', 'POROPL', 'IOO']},
 {'id': 9, 'cluster': ['DazDalee', 'vazalore', 'Daisleep']}]

In [37]:
string_clusters = pd.DataFrame.from_dict(clusters)


In [43]:
len(string_clusters)

10193

In [41]:
unclustered_brands = pd.DataFrame.from_dict(leftover_brands)


In [44]:
len(unclustered_brands)

2212

In [45]:
merged_df = pd.concat([unclustered_brands, string_clusters])

In [46]:
len(merged_df)

12405

In [55]:
brand_dict_path = '/home/preacher/Bungee/bnt/brand_dict/bd.csv'

In [62]:
brnad_dict = import_file_from_s3(s3_bucket, brand_dict_path)


In [63]:
len(brnad_dict)

662019

In [64]:
brnad_dict.rename(columns = {'crawled_name':'brand'}, inplace = True)

In [65]:
brnad_dict

,brand,image_url_1,product_url_1,product_url_2
0,JiangYanus,https://m.media-amazon.com/images/i/31da+jzj47...,https://amazon.com/dp/b0895m5hvz?th=1&psc=1,https://amazon.com/dp/b09vgzjt5s?th=1&psc=1
1,sofliym,https://m.media-amazon.com/images/i/71a6gi-q7k...,https://amazon.com/dp/b09hkpj496?th=1&psc=1,https://amazon.com/dp/b09hkpj496?th=1&psc=1
2,Hitching Post,https://d1s8987jlndkbs.cloudfront.net/assets/m...,https://acmemarkets.com/shop/product-details.9...,https://totalwine.com/wine/red-wine/syrahshira...
3,Snowball,https://d1s8987jlndkbs.cloudfront.net/assets/m...,https://amazon.com/dp/b0766j7xx5,https://totalwine.com/spirits/liqueurs-cordial...
4,Bully Boy,https://d2d8wwwkmhfcva.cloudfront.net/1200x/fi...,https://harristeeter.com/p/bully-boy-s-famous-...,https://totalwine.com/spirits/vodka/vodka/bull...
...,...,...,...,...
662014,Richard Alan Bunch,NaN,NaN,NaN
662015,jie Rui,NaN,NaN,NaN
662016,jdfkdskndvkrc,NaN,NaN,NaN
662017,Great Value Greek Style Yogurt Cranberries,NaN,NaN,NaN


In [61]:
merged_df

,id,brand
0,-1,G.H. MUMM
1,-1,FCXJTU
2,-1,WhimsicalDarkness
3,-1,CALWISE SPIRITS CO
4,-1,Hyde and Eek
...,...,...
10188,3788,ATWATEC
10189,3788,SMARTWATER
10190,3789,PONTMENT
10191,3789,LONGTEN


In [71]:
db_format_result_df = unmatched_brands.set_index('brand').join(merged_df.set_index('brand'), on = 'brand' , how='inner', lsuffix='_bd', rsuffix='_sc')

In [72]:
db_format_result_df

,image_url_1,product_url_1,product_url_2,id
brand,,,,
Snowball,https://d1s8987jlndkbs.cloudfront.net/assets/m...,https://amazon.com/dp/b0766j7xx5,https://totalwine.com/spirits/liqueurs-cordial...,2018
Danedvi,https://m.media-amazon.com/images/i/510wjx2uda...,https://amazon.com/dp/b091dypsn6,https://amazon.com/dp/b0b1mt389p,1770
IDEAL PET PRODUCTS SINCE 1979,https://i5.walmartimages.com/asr/c13a6f39-a629...,https://amazon.ca/dp/b000tvsxi6?th=1&psc=1,https://walmart.ca/en/ip/perfect-pet-multi-fle...,81
Sailor Jerry,https://assets-prd-ray.unataops.com/web/produc...,https://amazon.com/dp/b005hqnyxw,https://walmart.com/ip/44391632,1788
Redd's,NaN,https://acmemarkets.com/shop/product-details.9...,https://walgreens.com/store/c/redd's-peach-ale...,1808
...,...,...,...,...
TILIANG,NaN,NaN,NaN,1295
VinoDrip,NaN,NaN,NaN,-1
Value-Pak,https://assets.wakefern.com/is/image/wakefern/...,https://shop.shoprite.com/store/7330772/produc...,https://shoprite.com/sm/pickup/rsid/218/produc...,1261


In [69]:
# Python3 code to demonstrate
# convert list of tuples to list of list
# using list comprehension
 
# initializing list
test_list = [(1), (3), (5)]
 
# printing original list
print("The original list of tuples : " + str(test_list))
 
# using list comprehension
# convert list of tuples to list of list
res = [ele for ele in test_list]
 
# print result
print("The converted list of list : " + str(res))

The original list of tuples : [1, 3, 5]
The converted list of list : [1, 3, 5]


In [74]:
sample_df = db_format_result_df.iloc[:100]

In [75]:
sample_df

,image_url_1,product_url_1,product_url_2,id
brand,,,,
Snowball,https://d1s8987jlndkbs.cloudfront.net/assets/m...,https://amazon.com/dp/b0766j7xx5,https://totalwine.com/spirits/liqueurs-cordial...,2018
Danedvi,https://m.media-amazon.com/images/i/510wjx2uda...,https://amazon.com/dp/b091dypsn6,https://amazon.com/dp/b0b1mt389p,1770
IDEAL PET PRODUCTS SINCE 1979,https://i5.walmartimages.com/asr/c13a6f39-a629...,https://amazon.ca/dp/b000tvsxi6?th=1&psc=1,https://walmart.ca/en/ip/perfect-pet-multi-fle...,81
Sailor Jerry,https://assets-prd-ray.unataops.com/web/produc...,https://amazon.com/dp/b005hqnyxw,https://walmart.com/ip/44391632,1788
Redd's,NaN,https://acmemarkets.com/shop/product-details.9...,https://walgreens.com/store/c/redd's-peach-ale...,1808
...,...,...,...,...
Revlon Consumer Products Corp.,https://dsom-imager-prod.shipt.com/3943567/79e...,https://acmemarkets.com/shop/product-details.9...,https://shop.shipt.com/products/3943567,1799
Sure-Life Products,NaN,NaN,NaN,3371
DJS,https://m.media-amazon.com/images/i/5139pde+ev...,https://amazon.com/dp/b08r6yg7nf,https://amazon.com/dp/b0b2pd6pgq,1748


In [81]:
update_data = sample_df.iloc[:50]

In [82]:
update_data = 

,image_url_1,product_url_1,product_url_2,id
brand,,,,
Snowball,https://d1s8987jlndkbs.cloudfront.net/assets/m...,https://amazon.com/dp/b0766j7xx5,https://totalwine.com/spirits/liqueurs-cordial...,2018
Danedvi,https://m.media-amazon.com/images/i/510wjx2uda...,https://amazon.com/dp/b091dypsn6,https://amazon.com/dp/b0b1mt389p,1770
IDEAL PET PRODUCTS SINCE 1979,https://i5.walmartimages.com/asr/c13a6f39-a629...,https://amazon.ca/dp/b000tvsxi6?th=1&psc=1,https://walmart.ca/en/ip/perfect-pet-multi-fle...,81
Sailor Jerry,https://assets-prd-ray.unataops.com/web/produc...,https://amazon.com/dp/b005hqnyxw,https://walmart.com/ip/44391632,1788
Redd's,NaN,https://acmemarkets.com/shop/product-details.9...,https://walgreens.com/store/c/redd's-peach-ale...,1808
Muscle Feast,https://m.media-amazon.com/images/i/6198jpy9m7...,https://amazon.com/dp/b00506v814,https://amazon.com/dp/b09k4mv7s8,2901
Rock Garden,https://d2lnr5mha7bycj.cloudfront.net/product-...,https://amazon.com/dp/b003p405ey,https://shipt.com/shop/products/7745845,16
Gonipol,https://m.media-amazon.com/images/i/51-pkjcm3n...,https://amazon.com/dp/b09ndm1lhk?th=1&psc=1,https://amazon.com/dp/b09vc48l18?th=1&psc=1,1784
Harpily,NaN,NaN,NaN,2172


In [85]:
brand_list = update_data.image_url_1.values.tolist()

In [88]:
update_data.reset_index(inplace=True)

In [91]:
brand_list = update_data.brand.values.tolist()

In [92]:
brand_list

['Snowball',
 'Danedvi',
 'IDEAL PET PRODUCTS SINCE 1979',
 'Sailor Jerry',
 "Redd's",
 'Muscle Feast',
 'Rock Garden',
 'Gonipol',
 'Harpily',
 'Perfect Duster',
 'Johnson',
 'Monk',
 'GARNETIN',
 'Organic Way',
 'CHATEAU MICHEL DE VERT',
 'Secret for Longevity',
 'Z-Flex',
 'EARTH FRIENDLY',
 'Golden Care',
 'NOTHERS',
 'Solo Brands/Sokol and Company',
 'Mally',
 'No Boundaries',
 'Uwilowe',
 'Top-Tech',
 'Herchr',
 'Abbott Animal Health',
 'Grandstream',
 'K',
 'Berwick',
 '7UP',
 'Great American',
 'Samuel Adams',
 'Amzey',
 'Womens Tops',
 '2 Scoops',
 'HJBhjb',
 'Wokyo',
 'GLEEmade',
 'PetPortraitArtStudio',
 'Jinxuny',
 'SHUNSHENG',
 'Climature',
 'night shift brewing',
 'The Artful Canine',
 'The Bug Company',
 'Untitled Art',
 'Elite Home Products',
 'ROSEBEAR',
 'Rising Moon Organic']

In [96]:
sample_df.reset_index(inplace=True)

In [98]:
updation_brands = sample_df[sample_df['brand'].isin(brand_list)]
insertion_brands = sample_df[~sample_df['brand'].isin(brand_list)]

In [99]:
updation_brands

,brand,image_url_1,product_url_1,product_url_2,id
0,Snowball,https://d1s8987jlndkbs.cloudfront.net/assets/m...,https://amazon.com/dp/b0766j7xx5,https://totalwine.com/spirits/liqueurs-cordial...,2018
1,Danedvi,https://m.media-amazon.com/images/i/510wjx2uda...,https://amazon.com/dp/b091dypsn6,https://amazon.com/dp/b0b1mt389p,1770
2,IDEAL PET PRODUCTS SINCE 1979,https://i5.walmartimages.com/asr/c13a6f39-a629...,https://amazon.ca/dp/b000tvsxi6?th=1&psc=1,https://walmart.ca/en/ip/perfect-pet-multi-fle...,81
3,Sailor Jerry,https://assets-prd-ray.unataops.com/web/produc...,https://amazon.com/dp/b005hqnyxw,https://walmart.com/ip/44391632,1788
4,Redd's,NaN,https://acmemarkets.com/shop/product-details.9...,https://walgreens.com/store/c/redd's-peach-ale...,1808
5,Muscle Feast,https://m.media-amazon.com/images/i/6198jpy9m7...,https://amazon.com/dp/b00506v814,https://amazon.com/dp/b09k4mv7s8,2901
6,Rock Garden,https://d2lnr5mha7bycj.cloudfront.net/product-...,https://amazon.com/dp/b003p405ey,https://shipt.com/shop/products/7745845,16
7,Gonipol,https://m.media-amazon.com/images/i/51-pkjcm3n...,https://amazon.com/dp/b09ndm1lhk?th=1&psc=1,https://amazon.com/dp/b09vc48l18?th=1&psc=1,1784
8,Harpily,NaN,NaN,NaN,2172
9,Perfect Duster,https://target.scene7.com/is/image/target/gues...,https://target.com/p/perfect-duster-power-dust...,https://target.com/p/perfect-duster-power-dust...,1892


In [100]:
insertion_brands

,brand,image_url_1,product_url_1,product_url_2,id
50,MERS,NaN,NaN,NaN,1071
51,Press Coff,https://assets-prd-spr.unataops.com/web/produc...,https://shop.sprouts.com/product/31431/press-c...,https://shop.sprouts.com/product/31434/press-c...,1812
52,UP4 Probiotics,https://d2lnr5mha7bycj.cloudfront.net/product-...,https://instacart.com/store/items/1392430736,https://tgtappdata.com/v2/products/pdp/tcin/50...,1798
53,FAREYY,https://m.media-amazon.com/images/i/619qqfe7ts...,https://amazon.com/dp/b087j9jnb4?th=1&psc=1,https://amazon.com/dp/b09pt96hxd,1772
54,Teachers,https://d2lnr5mha7bycj.cloudfront.net/product-...,https://instacart.com/store/items/100160675,https://totalwine.com/spirits/scotch/blended-s...,1820
55,LifeSeasons,NaN,https://centralmarket.com/product/lifeseasons-...,https://walmart.com/ip/118210500,1806
56,Dotpet,NaN,NaN,NaN,1763
57,3 Beer,https://d2lnr5mha7bycj.cloudfront.net/product-...,https://instacart.com/store/items/109030320737,https://kroger.com/p/3-beer-smokin-chipotle-ru...,1785
58,HEAYEE,https://m.media-amazon.com/images/i/318bek0qdc...,https://amazon.com/dp/b0bc98bbbn,https://amazon.com/dp/b0bhmvrpgv,1752
59,Fran's Chocolates,https://d2d8wwwkmhfcva.cloudfront.net/1200x/fi...,https://amazon.com/frans-dark-chocolate-sauce-...,https://shop.sprouts.com/product/78731/fran's-...,1113


In [101]:
 insertion_brands.to_csv( '/home/preacher/Bungee/bnt/insert.csv', index=False)